## Курсов проект
### По Общо Езикознание
#### на Тема: Анотиране на езикови данни (1.  Част на речта и лема; 2.  Собствени имена)
##### изготвил: Димитър Фенерски 4MI3400850

In [1]:
from typing import Literal, List, Tuple, cast, Dict
from collections import Counter
from transformers import (
    BertTokenizer,
    AutoModelForTokenClassification,
    BatchEncoding,
    TrainingArguments,
    DataCollatorForTokenClassification,
    Trainer,
)
from torch.utils.data import Dataset
import torch
import numpy as np

In [2]:
torch.cuda.is_available()

True

## Подготовка на тренировъчния корпус

Име: `UD_Bulgarian-BTB`

Съдържание: $156 149$ tokens ($11,138$ sentences)

Формат: `UniversalDependency`

In [3]:
PrimitiveDataset = List[Tuple[List[str], List[str]]]

NUM_TAGS = 16

TAG2IDX = {
    "ADP": 0,
    "NOUN": 1,
    "PUNCT": 2,
    "VERB": 3,
    "AUX": 4,
    "PRON": 5,
    "ADJ": 6,
    "PART": 7,
    "ADV": 8,
    "INTJ": 9,
    "DET": 10,
    "PROPN": 11,
    "CCONJ": 12,
    "NUM": 13,
    "SCONJ": 14,
    "X": 15,
}
IDX2TAG = {
    0: "ADP",
    1: "NOUN",
    2: "PUNCT",
    3: "VERB",
    4: "AUX",
    5: "PRON",
    6: "ADJ",
    7: "PART",
    8: "ADV",
    9: "INTJ",
    10: "DET",
    11: "PROPN",
    12: "CCONJ",
    13: "NUM",
    14: "SCONJ",
    15: "X",
}

In [10]:
def parse_dataset(
    dataset: Literal["train"] | Literal["dev"] | Literal["test"],
) -> PrimitiveDataset:
    assert dataset in ["train", "dev", "test"]

    tokens = []

    with open(f"./corpus/bg_btb-ud-{dataset}.conllu") as file:
        sents = file.read().split("\n" * 2)
        for sent in sents:
            if not sent:
                continue

            sent_words = []
            sent_pos_types = []

            rows = sent.split("\n")
            for r in rows:
                if r[0] == "#":
                    continue
                _, word, _, pos_type, *_ = r.split("\t")
                sent_words.append(word)
                sent_pos_types.append(pos_type)

            tokens.append((sent_words, sent_pos_types))

    return tokens

In [4]:
def count_tokens(dataset: PrimitiveDataset) -> Counter:
    tokens = [token for (_, sent_tokens) in dataset for token in sent_tokens]
    return Counter(tokens)

## Подравняване

Моделът, който ще бъде използван за обучението е `google-bert/bert-base-multilingual-cased`.
Това е базов многоезичен `BERT` модел, сензитивен към casing-а, позволяващ донатрениране/фина настройка/fine-tuning

При предварителната подготовка на модела е използвана `WordPiece` токенизация с общ речник възлизащ на $110,000$ думи.
Моделът е трениран върху $104$-те езика с най-големи `Wikipedia`-корпуси, сред които и български.

За използването му за класификация за части на речта е необходима процедура на подравняване и запълване, тъй като думи, разбити на поддуми от `WordPiece` следва да бъдат анотирани само веднъж.

In [5]:
def tokenize_and_align(sent_words, sent_tags, tokenizer) -> Dict:
    T = tokenizer(
        sent_words,
        is_split_into_words=True,
        max_length=128,
        truncation=True,
        padding="max_length",
    )
    # T = cast(BatchEncoding, T)
    word_ids = T.word_ids()
    padded_batch_encoding = {
        "input_ids": [],
        "attention_mask": [],
        "labels": [],
    }

    for i, (ii, tti, am) in enumerate(
        zip(
            T["input_ids"],
            T["token_type_ids"],
            T["attention_mask"],
        )
    ):
        padded_batch_encoding["input_ids"].append(ii)
        padded_batch_encoding["attention_mask"].append(am)

        padded_batch_encoding["labels"].append(
            TAG2IDX[sent_tags[word_ids[i]]]
            if word_ids[i] is not None  # skip special tokens
            and word_ids[i] != word_ids[i - 1]  # only first tokens get a tag
            #and i < len(sent_tags)  # prevent overflows
            else -100  # the special ignore token of `CrossEntropyLoss` in pytorch
        )

    return padded_batch_encoding

## Трениране

Използваме предварително разделените на тренировъчен и тестови пакет данни за обучението.
При експериментиране се потвърждава, че дори и нисък брой тренировъчни периоди ($3$) са достатъчни за постигане на $99\%+$ точност, след което моделът започва да се пренапасва.


In [6]:
class POSDataset(Dataset):
    def __init__(self, pds: PrimitiveDataset, tokenizer):
        self.tokenizer = tokenizer
        self.pds = pds
        self.tds = []
        for sent_words, sent_tokens in self.pds:
            pbe = tokenize_and_align(sent_words, sent_tokens, self.tokenizer)
            self.tds.append(
                {
                    "input_ids": torch.tensor(pbe["input_ids"]),
                    "attention_mask": torch.tensor(pbe["attention_mask"]),
                    "labels": torch.tensor(pbe["labels"]),
                }
            )

    def __len__(self):
        return len(self.tds)

    def __getitem__(self, idx):  # type: ignore (the LSP complains)
        return self.tds[idx]

In [7]:
def compute_metrics(eval_prediction):
    (predictions, label_ids) = eval_prediction
    predictions = np.argmax(predictions, axis=2)

    total = 0
    correct = 0

    for i in range(len(predictions)):
        compare_tuples = [t for t in zip(predictions[i], label_ids[i]) if t[1] != -100]
        total = total + len(compare_tuples)
        correct = correct + sum(1 for t in compare_tuples if t[0] == t[1])

    return {"accuracy": correct / total}

In [8]:
def create_trainer(model, train_dataset, test_dataset, tokenizer):
    # https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
    training_args = TrainingArguments(
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        learning_rate=2e-5,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
        warmup_ratio=0.1,
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        processing_class=tokenizer,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics,
    )

    return trainer

In [12]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer = cast(BertTokenizer, tokenizer)
train_dataset = POSDataset(parse_dataset("train"), tokenizer)
test_dataset = POSDataset(parse_dataset("test"), tokenizer)
# print(dataset[0])
# print(dataset[0]["labels"].size())
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=NUM_TAGS,
    id2label=IDX2TAG,
    label2id=TAG2IDX,
)
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
trainer = create_trainer(model, train_dataset, test_dataset, tokenizer)
trainer.train()

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

BertForTokenClassification LOAD REPORT from: bert-base-multilingual-cased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.seq_relationship.bias                  | UNEXPECTED | 
bert.pooler.dense.weight                   | UNEXPECTED | 
bert.pooler.dense.bias                     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly ini

Total parameters: 177275152


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

TrainOutput(global_step=1671, training_loss=0.17145288868053168, metrics={'train_runtime': 313.9728, 'train_samples_per_second': 85.106, 'train_steps_per_second': 5.322, 'total_flos': 1745748596109312.0, 'train_loss': 0.17145288868053168, 'epoch': 3.0})

In [13]:
trainer.save_model("output")
tokenizer.save_pretrained("output")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('output/tokenizer_config.json', 'output/tokenizer.json')

## Оценка на получените резултати

Ще демонстрираме способносите на модела с няколко текстови изречения.
Оценката от $99\%+$ при трениране показва, че моделът лесно се справя с книжовен български език.

### Диалектни текстове

Поради многоезичността на модела, сходните писменост, морфология и речник, виждаме и трансфер на знания и към македонски.
Можем да очакваме сходен резултат и за диалектни текстове.
Тук сме възпрепятствани от липсата на "добри" / анотирани диалектни корпуси.

### Разговорни текстове

За разговорни текстове задачата малко се усложнява поради наличието на голям брой думи, които моделът никога не е виждал -
разговорната реч не се среща често в Уикипедия.

Резултатите се влошават значително, ако тестваме върху "шльокавица" - това е на практика напълно непознат за модела текст.

#### Възможни подходи за решение

Чрез фиксиран набор от правила за превод към шльокавица е възможно да се създаде синтетичен корпус, на базата на `UD_Bulgarian-BTB`, върху който отново да тренираме модела.

In [14]:
TEST_DATA = [
    "Късият път през планината е заснежен.",
    "Алчната за пари оперна певица купи на продуцента нов бял мерцедес с вертикални врати?",
    "kak ti e feisbooka",
    "Kvo 6e praim sled daskalo",
    "Заедно со Соња, тој оди во полициската станица и пред Иља Петрович го признава злосторството што го направил. (Не сака тоа да го стори пред Порфириј, зашто го мрази неговиот цинизам и му е смачено целото негово иследување)."
    "6te hodim li, 4e zamruzvam",
]

In [15]:
def predict(text: str, model, tokenizer):
    idx2tag = model.config.id2label
    words = text.split()
    tokens = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
    )

    word_ids = tokens.word_ids()

    res_p = []

    model.to("cpu")
    model.eval()
    with torch.no_grad():
        logits = model(**tokens).logits
        [predictions] = torch.argmax(logits, dim=2)

        for i, p in enumerate(predictions):
            if word_ids[i] is not None and word_ids[i] != word_ids[i - 1]:
                res_p.append(idx2tag[p.item()])

    return list(zip(words, res_p))


def print_prediction(prediction: List[Tuple[str, str]]):
    for p in prediction:
        print(f"{p[0]}\t{p[1]}")


def run_predictions(model, tokenizer):
    for test_sent in TEST_DATA:
        prediction = predict(
            test_sent,
            model,
            tokenizer,
        )
        print_prediction(prediction)
        print("-" * 20)

In [16]:
from torch.utils.data import DataLoader
from collections import defaultdict


def eval_total_accuracy(test_dataset, model):
    dataloader = DataLoader(test_dataset, batch_size=32)

    total = 0
    correct = 0

    model.to("cpu")
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            logits = model(**batch).logits
            predictions = torch.argmax(logits, dim=2)
            mask = batch["labels"] != -100
            correct = correct + (predictions == batch["labels"])[mask].sum().item()
            total = total + mask.sum().item()

    return correct / total


def eval_per_tag_accuracy(test_dataset, model):
    dataloader = DataLoader(test_dataset, batch_size=32)

    total = defaultdict(int)
    correct = defaultdict(int)
    confusion_matrix = torch.zeros((NUM_TAGS, NUM_TAGS))

    model.to("cpu")
    model.eval()

    with torch.no_grad():
        for batch in dataloader:
            logits = model(**batch).logits
            predictions = torch.argmax(logits, dim=2)
            mask = batch["labels"] != -100
            for pred, gold in zip(predictions[mask], batch["labels"][mask]):
                if pred.item() == gold.item():
                    correct[gold.item()] += 1

                total[gold.item()] += 1

                confusion_matrix[gold.item()][pred.item()] += 1

    # return total, correct, confusion_matrix

    total_labeled = {
        IDX2TAG[k]:v for (k,v) in total.items()
    }
    correct_labeled = {
        IDX2TAG[k]:v for (k,v) in correct.items()
    }

    return total_labeled, correct_labeled, confusion_matrix

In [18]:
model_path = "./output"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

run_predictions(model, tokenizer)
test_dataset = POSDataset(parse_dataset("test"), tokenizer)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

Късият	ADJ
път	NOUN
през	ADP
планината	NOUN
е	AUX
заснежен.	VERB
--------------------
Алчната	ADJ
за	ADP
пари	NOUN
оперна	ADJ
певица	NOUN
купи	VERB
на	ADP
продуцента	NOUN
нов	ADJ
бял	ADJ
мерцедес	NOUN
с	ADP
вертикални	ADJ
врати?	NOUN
--------------------
kak	PART
ti	PART
e	AUX
feisbooka	NOUN
--------------------
Kvo	ADV
6e	NUM
praim	VERB
sled	ADJ
daskalo	NOUN
--------------------
Заедно	ADV
со	ADP
Соња,	PROPN
тој	PRON
оди	VERB
во	ADP
полициската	ADJ
станица	NOUN
и	CCONJ
пред	ADP
Иља	PROPN
Петрович	PROPN
го	PRON
признава	VERB
злосторството	NOUN
што	PRON
го	PRON
направил.	VERB
(Не	PUNCT
сака	VERB
тоа	PRON
да	AUX
го	PRON
стори	VERB
пред	ADP
Порфириј,	PROPN
зашто	SCONJ
го	PRON
мрази	VERB
неговиот	DET
цинизам	NOUN
и	CCONJ
му	PRON
е	AUX
смачено	VERB
целото	ADJ
негово	DET
иследување).6te	NOUN
hodim	NOUN
li,	NOUN
4e	ADJ
zamruzvam	NOUN
--------------------


In [19]:
test_dataset = POSDataset(parse_dataset("test"), tokenizer)
total_accuracy = eval_total_accuracy(test_dataset, model)
print(total_accuracy)

0.9902653178087422


In [20]:
*_, confusion_matrix = eval_per_tag_accuracy(test_dataset, model)
print(confusion_matrix)

tensor([[2.2340e+03, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 3.4440e+03, 0.0000e+00, 6.0000e+00, 0.0000e+00, 2.0000e+00,
         1.0000e+01, 0.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00, 1.9000e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.2680e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 3.0000e+00, 0.0000e+00, 1.6400e+03, 1.0000e+00, 1.0000e+00,
         6.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 9.1000e+02, 4.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000